In [ ]:
import os
import tensorflow as tf


In [ ]:
os.chdir("../")

In [ ]:
model= tf.keras.models.load_model("artifacts/training/model.h5")

In [ ]:
model.summary()

In [ ]:
##Setting up environment variables for mlflow
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Vikaslakkacs/First_classifier.mlflow "
os.environ["MLFLOW_TRACKING_USERNAME"]="Vikaslakkacs"
os.environ["MLFLOW_TRACKING_PASSWORD"]="7af2883531f922b7f10037acc2db95ccdd02bfa5"

In [ ]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params: dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int
    

In [ ]:
from firstClassifier.constants import *
from firstClassifier.utils import read_yaml, create_directories
class ConfigurationManager:

    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH
             ):
        """_summary_: Creates artifact directories taken from config.yaml file
        """
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        ##Create directories
        create_directories([self.config.artifacts_root])
    
    def get_validation_config(self)->EvaluationConfig:
        eval_config= EvaluationConfig(
            path_of_model= "artifacts/training/model.h5",
            training_data= "artifacts/data_ingestion/PetImages",
            all_params= self.params,
            mlflow_uri=self.config.evaluation.mlflow_uri,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from pathlib import Path
from firstClassifier.utils import save_json
import mlflow
import mlflow.tensorflow
from urllib.parse import urlparse
#from firstClassifier.entity import EvaluationConfig


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config= config

    
    def _valid_generator(self):

        ##Augmentation
        datagenerator_kwargs= dict(
            rescale= 1./255,
            validation_split= 0.005

        )

        dataflow_kwargs= dict(
            target_size= self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation= "bilinear"
        )

        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator= valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle= False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

        
    def evaluation(self):
        self.model= self.load_model(self.config.path_of_model)
        self._valid_generator()
        ##score returns loss and accuracy
        self.score= model.evaluate(
            self.valid_generator
        )

    def save_score(self):
        scores= {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data= scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        #print("f{self.config.mlflow_uri}-----Hello")
        tracking_url_type_store= urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            mlflow.log_params(
                self.config.all_params
            )
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store!="file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16 model")
            else:
                mlflow.keras.log_model(self.model, "model")




In [ ]:
try:
    config= ConfigurationManager()
    validation_config= config.get_validation_config()
    evaluate= Evaluation(validation_config)
    evaluate.evaluation()
    evaluate.save_score()
    evaluate.log_into_mlflow()

except Exception as e:
    raise e

In [ ]:

!mlflow ui